In [13]:
inp = open('day9.txt', 'r').read()

In [14]:
from itertools import combinations
boxes = [tuple(map(int, line.split(','))) for line in inp.splitlines()]


max((abs(x1 - x2) + 1) * (abs(y1 - y2) + 1) for (x1, y1), (x2, y2) in combinations(boxes, 2))

4763040296

In [15]:
from itertools import combinations
import bisect
from tqdm import tqdm
from functools import cache

horiz = []
verts = []
for (x1, y1), (x2, y2) in zip(boxes, boxes[1:]+boxes[:1]):
    if y1 == y2:
        horiz.append((min(x1, x2), max(x1, x2), y1))
    else:
        verts.append((x1, min(y1, y2), max(y1, y2)))

horiz.sort()
verts = set(verts)
@cache
def valid_intervals(x):
    intervals = []
    for x1, x2, y in horiz:
        if x1 <= x <= x2:
            intervals.append((x1, x2, y))
    intervals.sort(key=lambda t: t[2])

    valid = []
    out = False
    for i1, i2 in zip(intervals, intervals[1:]):
        x1, x2, y1 = i1
        x3, x4, y2 = i2
        if x2 == x3 and (x2, y1, y2) in verts:
            valid.append((y1, y2))
        elif x1 == x4 and (x1, y1, y2) in verts:
            valid.append((y1, y2))
        else:
            if not out:
                valid.append((y1, y2))
            out = not out
    return valid

box_pairs = sorted(combinations(boxes, 2), 
                   key=lambda pair: (abs(pair[0][0] - pair[1][0]) + 1) * (abs(pair[0][1] - pair[1][1]) + 1),
                   reverse=True)

max_area = 0
for (x1, y1), (x2, y2) in tqdm(box_pairs):
    area = (abs(x1 - x2) + 1) * (abs(y1 - y2) + 1)
    
    if x2 < x1:
        (x1, y1), (x2, y2) = (x2, y2), (x1, y1)
    
    ok = True
    for x in range(x1, x2 + 1):
        valid = valid_intervals(x)
        
        y_low = min(y1, y2)
        y_high = max(y1, y2)
        
        idx = bisect.bisect_right(valid, (y_low, float('inf'))) - 1
        
        if idx == -1 or not (valid[idx][0] <= y_low and valid[idx][1] >= y_high):
            ok = False
            break
    
    if ok:
        max_area = area
        break

max_area

 43%|████▎     | 52526/122760 [03:04<04:07, 283.96it/s]  


1396494456